<a href="https://colab.research.google.com/github/OpenXRF/lead-screening/blob/main/Fig4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from pathlib import Path
import time

# ==========================================================
# Figure: Detector Parameter Effects on LOD (Python version)
# ==========================================================

np.random.seed(0)  # Reproduzierbarkeit

# ---------------- Pfade und Konzentrationen ----------------
print("Reading simulated data for 100 and 200 ppm...\n")

c = [100, 200]  # [ppm]
base_path = Path("../data/Fig3")

data100ppm = {}
data200ppm = {}

t0 = time.time()

print(" - 100 ppm data...")
for s in range(7):
    field = f"S{s}"
    fname = base_path / f"Am_19775890_2.5_100_{s}.csv"
    data100ppm[field] = pd.read_csv(fname)

print(" - 200 ppm data...")
for s in range(7):
    field = f"S{s}"
    fname = base_path / f"Am_19775890_2.5_200_{s}.csv"
    data200ppm[field] = pd.read_csv(fname)

time_read = time.time() - t0
print(f"Done! Took {time_read:.0f} seconds")

# ---------------- Preprocessing ----------------
print("\nPre-processing data...")
t1 = time.time()

FWHM_keV_base = 0.15  # [keV]
detector_area_base = 50.0  # [mm²]
detector_radius_base = np.sqrt(detector_area_base / np.pi)

binwidth = 0.03  # [keV]
E = np.arange(0, 75 + binwidth, binwidth)

# Peak-Parameter
peak_alpha_E = 10.55  # [keV]
peak_beta_E = 12.65   # [keV]

# Parameter-Ranges
resolution_values = [0.15, 0.25, 0.35, 0.50]  # [keV]
efficiency_values = [1.00, 0.75, 0.50, 0.25]
datasets_base = [data100ppm, data200ppm]

# preprocessed[res_idx][conc_idx] -> Dict mit S0..S6, alpha_window, beta_window
preprocessed = [[None for _ in c] for _ in resolution_values]

for res_idx, FWHM in enumerate(resolution_values):
    peak_window = FWHM
    alpha_window = (E >= peak_alpha_E - peak_window) & (E <= peak_alpha_E + peak_window)
    beta_window = (E >= peak_beta_E - peak_window) & (E <= peak_beta_E + peak_window)

    for conc_idx, data_conc in enumerate(datasets_base):
        entry = {"alpha_window": alpha_window, "beta_window": beta_window}
        for s in range(7):
            field = f"S{s}"
            df = data_conc[field]

            # Energieauflösung
            df["E"] = df["Energy(MeV)"] * 1000.0 + (FWHM / 2.355) * np.random.randn(len(df))

            # radialer Treffer
            df["r_hit"] = np.sqrt(df["x(mm)"]**2 + df["y(mm)"]**2)

            # Effizienzmasken
            for eff_idx, eff_val in enumerate(efficiency_values):
                mask_name = f"eff_mask_{eff_idx+1}"
                df[mask_name] = np.random.rand(len(df)) <= eff_val

            entry[field] = df

        preprocessed[res_idx][conc_idx] = entry

time_preprocess = time.time() - t1
print(f"Pre-processing done! Took {time_preprocess:.0f} seconds")

# ---------------- Zeitdiskretisierung ----------------
fluence = 995 * np.pi * 2 * (1 - np.cos(np.pi / 4))  # [ph/s]
step_size_min = 5  # [min]
max_event = 180 * 60 * fluence
step_size = step_size_min * 60 * fluence
num_steps = int(np.floor(max_event / step_size))
time_values = np.arange(1, num_steps + 1) * step_size_min  # [min]

# ---------------- Parameter-Variationen ----------------
area_values = [50.0, 40.0, 25.0, 12.5]  # [mm²]
detector_radii = np.sqrt(np.array(area_values) / np.pi)

LOD_results = {
    "efficiency": np.zeros((len(c), len(efficiency_values))),
    "efficiency_err": np.zeros((len(c), len(efficiency_values))),
    "resolution": np.zeros((len(c), len(resolution_values))),
    "resolution_err": np.zeros((len(c), len(resolution_values))),
    "area": np.zeros((len(c), len(area_values))),
    "area_err": np.zeros((len(c), len(area_values))),
}

# ==========================================================
#   Hilfsfunktion: LOD-Berechnung (Python-Version calculateLOD_optimized)
# ==========================================================

def calculateLOD_optimized(data_struct, eff_idx, detector_radius,
                           time_values, num_steps, step_size,
                           E, binwidth):
    alpha_window = data_struct["alpha_window"]
    beta_window = data_struct["beta_window"]

    signal_over_time = np.zeros(num_steps, dtype=float)
    background_over_time = np.zeros(num_steps, dtype=float)
    SNR_combined = np.zeros(num_steps, dtype=float)

    eff_field = f"eff_mask_{eff_idx+1}"

    # Vorfilterung für alle sieben Quellen
    filtered_data = []
    for s in range(7):
        field = f"S{s}"
        df = data_struct[field]
        base_mask = (
            df[eff_field] &
            (df["Particle"] == "gamma") &
            (df["r_hit"] <= detector_radius)
        )
        filtered_data.append(df.loc[base_mask])

    # Zeit-Schritte
    for step in range(num_steps):
        end_event = (step + 1) * step_size

        total_alpha_secondary = 0
        total_alpha_primary = 0
        total_beta_secondary = 0
        total_beta_primary = 0

        for s in range(7):
            df = filtered_data[s]
            event_mask = df["EventID"] < end_event
            data_step = df.loc[event_mask]

            if data_step.empty:
                continue

            is_primary = data_step["Type"] == "Primary"
            primary_E = data_step.loc[is_primary, "E"].to_numpy()
            secondary_E = data_step.loc[~is_primary, "E"].to_numpy()

            primary_hist, _ = np.histogram(primary_E, bins=np.append(E, E[-1] + binwidth))
            secondary_hist, _ = np.histogram(secondary_E, bins=np.append(E, E[-1] + binwidth))

            total_alpha_primary += primary_hist[alpha_window].sum()
            total_alpha_secondary += secondary_hist[alpha_window].sum()
            total_beta_primary += primary_hist[beta_window].sum()
            total_beta_secondary += secondary_hist[beta_window].sum()

        total_secondary = total_alpha_secondary + total_beta_secondary
        total_primary = total_alpha_primary + total_beta_primary

        signal_over_time[step] = total_secondary
        background_over_time[step] = total_primary

        if (total_secondary + total_primary) > 0:
            SNR_combined[step] = total_secondary / np.sqrt(total_secondary + total_primary)
        else:
            SNR_combined[step] = 0.0

    # Fit SNR(t) = A * sqrt(t)
    def sqrt_model(t, A):
        return A * np.sqrt(t)

    # Nur Punkte mit t>0 verwenden
    t_data = np.asarray(time_values, dtype=float)
    y_data = np.asarray(SNR_combined, dtype=float)

    popt, _ = curve_fit(sqrt_model, t_data, y_data, p0=[1.0], maxfev=10000)
    A_value = popt[0]

    t_LOD = 9.0 / A_value**2  # SNR = 3 → t = 9/A²

    # Signal- und Hintergrundraten (linear fit)
    def linear_model(t, A):
        return A * t

    s_fit, _ = curve_fit(linear_model, t_data, signal_over_time, p0=[1.0], maxfev=10000)
    b_fit, _ = curve_fit(linear_model, t_data, background_over_time, p0=[1.0], maxfev=10000)

    s = s_fit[0]
    b = b_fit[0]
    t = t_LOD

    sigma_t = 9.0 * np.sqrt(((s + 2 * b)**2 / s**5) * (1.0 / t) + b / s**4 * (1.0 / t))
    return t_LOD, sigma_t

# ==========================================================
#   1. Effekt der Detektionseffizienz
# ==========================================================

print("\n=== Evaluating Detection Efficiency ===")
t2 = time.time()

for eff_idx, eff_val in enumerate(efficiency_values):
    print(f"Processing efficiency: {eff_val*100:.0f}%")
    for conc_idx, conc_val in enumerate(c):
        data_struct = preprocessed[0][conc_idx]  # Baseline-Auflösung (index 0 → 0.15 keV)
        t_lod, sigma = calculateLOD_optimized(
            data_struct, eff_idx,
            detector_radius_base,
            time_values, num_steps, step_size,
            E, binwidth
        )
        LOD_results["efficiency"][conc_idx, eff_idx] = t_lod
        LOD_results["efficiency_err"][conc_idx, eff_idx] = sigma

time_eff = time.time() - t2
print(f"Efficiency evaluation done! Took {time_eff:.0f} seconds")

# ==========================================================
#   2. Effekt der spektralen Auflösung
# ==========================================================

print("\n=== Evaluating Spectral Resolution ===")
t3 = time.time()

for res_idx, res_val in enumerate(resolution_values):
    print(f"Processing resolution: {res_val*1000:.0f} eV")
    for conc_idx, conc_val in enumerate(c):
        data_struct = preprocessed[res_idx][conc_idx]
        t_lod, sigma = calculateLOD_optimized(
            data_struct, 0,  # eff_idx = 0 → 100%
            detector_radius_base,
            time_values, num_steps, step_size,
            E, binwidth
        )
        LOD_results["resolution"][conc_idx, res_idx] = t_lod
        LOD_results["resolution_err"][conc_idx, res_idx] = sigma

time_res = time.time() - t3
print(f"Resolution evaluation done! Took {time_res:.0f} seconds")

# ==========================================================
#   3. Effekt der Detektorfläche
# ==========================================================

print("\n=== Evaluating Detector Area ===")
t4 = time.time()

for area_idx, area in enumerate(area_values):
    print(f"Processing area: {area:.1f} mm²")
    for conc_idx, conc_val in enumerate(c):
        data_struct = preprocessed[0][conc_idx]  # baseline resolution & eff
        t_lod, sigma = calculateLOD_optimized(
            data_struct, 0,  # 100% Effizienz
            detector_radii[area_idx],
            time_values, num_steps, step_size,
            E, binwidth
        )
        LOD_results["area"][conc_idx, area_idx] = t_lod
        LOD_results["area_err"][conc_idx, area_idx] = sigma

time_area = time.time() - t4
print(f"Area evaluation done! Took {time_area:.0f} seconds")

# ==========================================================
#   Laufzeit-Übersicht
# ==========================================================

print("\n=== Computation Time Summary ===")
print(f"Data reading: {time_read:.1f} s")
print(f"Pre-processing: {time_preprocess:.1f} s")
print(f"Efficiency evaluation: {time_eff:.1f} s")
print(f"Resolution evaluation: {time_res:.1f} s")
print(f"Area evaluation: {time_area:.1f} s")
print(f"Total time: {time_read + time_preprocess + time_eff + time_res + time_area:.1f} s")

# ==========================================================
#   Plotten – möglichst MATLAB-identisch
# ==========================================================

plt.rcParams.update({
    "font.size": 6,
    "font.family": "Arial",  # wenn du DM Sans installiert hast, kannst du hier "DM Sans" setzen
})

fig, axes = plt.subplots(1, 3, figsize=(20/2.54, 10/2.54))
fig.patch.set_facecolor("white")

baseline_eff = LOD_results["efficiency"][:, 0]
baseline_res = LOD_results["resolution"][:, 0]
baseline_area = LOD_results["area"][:, 0]

colors = plt.cm.tab10(np.arange(len(c)))

# 1) Detection Efficiency
ax = axes[0]
for conc_idx, conc_val in enumerate(c):
    y = LOD_results["efficiency"][conc_idx, :]
    yerr = LOD_results["efficiency_err"][conc_idx, :]
    x = np.array(efficiency_values) * 100.0

    ax.errorbar(x, y, yerr=yerr,
                fmt="o:", lw=1, ms=4,
                markerfacecolor=colors[conc_idx],
                color=colors[conc_idx],
                capsize=0,
                label=f"{conc_val} ppm")
    ax.plot([25, 100], [baseline_eff[conc_idx]]*2,
            "--", color="0.5", linewidth=0.8)

    for i, xv in enumerate(x):
        factor = y[i] / baseline_eff[conc_idx] if baseline_eff[conc_idx] > 0 else np.nan
        ax.text(xv - 3, y[i] + 4,
                f"{factor:.1f}x",
                fontsize=8, ha="right")

ax.set_xlabel("Detection Efficiency (%)")
ax.set_ylabel("LOD Time (minutes)")
ax.set_title("Effect of Detection Efficiency")
ax.legend(loc="upper left", fontsize=6)
ax.set_xticks(sorted(np.array(efficiency_values) * 100.0))
ax.set_xlim(20, 105)
ax.set_ylim(0, 120)
ax.invert_xaxis()
ax.tick_params(direction="out")
ax.set_aspect("equal")

# 2) Spectral Resolution
ax = axes[1]
for conc_idx, conc_val in enumerate(c):
    y = LOD_results["resolution"][conc_idx, :]
    yerr = LOD_results["resolution_err"][conc_idx, :]
    x = np.array(resolution_values) * 1000.0

    ax.errorbar(x, y, yerr=yerr,
                fmt="o:", lw=1, ms=4,
                markerfacecolor=colors[conc_idx],
                color=colors[conc_idx],
                capsize=0,
                label=f"{conc_val} ppm")
    ax.plot([100, 550], [baseline_res[conc_idx]]*2,
            "--", color="0.5", linewidth=0.8)

    for i, xv in enumerate(x):
        factor = y[i] / baseline_res[conc_idx] if baseline_res[conc_idx] > 0 else np.nan
        ax.text(xv + 18, y[i] + 4,
                f"{factor:.1f}x",
                fontsize=8, ha="left")

ax.set_xlabel("Energy Resolution (eV)")
ax.set_ylabel("LOD Time (minutes)")
ax.set_title("Effect of Spectral Resolution")
ax.legend(loc="upper left", fontsize=6)
ax.set_xticks(sorted(np.array(resolution_values) * 1000.0))
ax.set_xlim(100, 550)
ax.set_ylim(0, 120)
ax.tick_params(direction="out")
ax.set_aspect("equal")

# 3) Detector Area
ax = axes[2]
for conc_idx, conc_val in enumerate(c):
    y = LOD_results["area"][conc_idx, :]
    yerr = LOD_results["area_err"][conc_idx, :]
    x = np.array(area_values)

    ax.errorbar(x, y, yerr=yerr,
                fmt="o:", lw=1, ms=4,
                markerfacecolor=colors[conc_idx],
                color=colors[conc_idx],
                capsize=0,
                label=f"{conc_val} ppm")
    ax.plot([10, 55], [baseline_area[conc_idx]]*2,
            "--", color="0.5", linewidth=0.8)

    for i, xv in enumerate(x):
        factor = y[i] / baseline_area[conc_idx] if baseline_area[conc_idx] > 0 else np.nan
        ax.text(xv - 1.8, y[i] + 4,
                f"{factor:.1f}x",
                fontsize=8, ha="right")

ax.set_xlabel("Detector Area (mm²)")
ax.set_ylabel("LOD Time (minutes)")
ax.set_title("Effect of Detector Area")
ax.legend(loc="upper left", fontsize=6)
ax.set_xticks(sorted(area_values))
ax.set_xlim(10, 55)
ax.set_ylim(0, 120)
ax.invert_xaxis()
ax.tick_params(direction="out")
ax.set_aspect("equal")

fig.suptitle(
    "Detector Parameter Effects on LOD Time\n"
    "Baseline: 7× Am241, 150 eV, 50 mm², 100% efficiency",
    fontsize=8
)

plt.tight_layout(rect=[0, 0, 1, 0.92])
fig.savefig("Fig4_detector_requirements.pdf", bbox_inches="tight", dpi=300)
plt.show()

print("\nSaved figure as Fig4_detector_requirements.pdf")
